In [ ]:
#instala as bibliotecas necessárias para a execução
!pip install pandas
!pip install seaborn
!pip install numpy
!pip install matplotlib
!pip install sklearn

In [ ]:
#importando as bibliotecas que serão utilizadas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import warnings

#serão ignoradas todas mensagens de warning (aviso) que aparecer no notebook
warnings.filterwarnings('ignore')

## Abrindo o arquivo com os dados

In [ ]:
arquivo_xls = pd.ExcelFile('data/teste_smarkio_Lbs.xls')
dados_analise_ml = pd.read_excel(arquivo_xls, 'Análise_ML')
dados_nlp = pd.read_excel(arquivo_xls, 'NLP')

In [ ]:
#esses dados serão utilizados para resolver as atividades de 1 a 4.
dados_analise_ml.head(5)

In [ ]:
#esses dados serão utilizados para resolver a atividade 5.
dados_nlp.head(5)

## Atividades

### 1- Análise exploratória dos dados utilizando estatística descritiva e inferencial,considerando uma, duas e/ou mais variáveis.

In [ ]:
#pred_class = classe que foi identificada pelo modelo
#probabilidade = a probabilidade da classe que o modelo identificou
#status = status da classificação de acordo com um especialista
#true_class = classe verdadeira. Se for nula, assumir o valor do pred_class
#se pred_class = true_class, então o modelo acertou a classe

In [ ]:
#nós temos um total de 643 dados (também podemos chamar de linhas) e 4 colunas
dados_analise_ml.shape

In [ ]:
#primeiramente iremos analisar os tipos das colunas que temos
#'pred_class', 'probabilidade' e 'true_class' são colunas numéricas
#'status' é do tipo object. Ou seja, se trata de uma coluna categórica
dados_analise_ml.info()

In [ ]:
dados_analise_ml.describe(include='all') #iremos incluir todas as colunas, até mesmo as categóricas (status)

#a única coluna que possui valores nulos é a 'true_class', elas serão tratadas posteriormente
#a coluna 'status' só possui dois valores possíveis

In [ ]:
#analisando a coluna 'status'

fig = plt.figure(figsize=(6, 4))
plt.title('Análise da coluna Status')
sns.countplot(dados_analise_ml['status'])
fig.savefig('images/analise_status.png')

#a coluna 'status' possui dois tipos de valores: approved e revision
#approved é o valor que aparece com mais frequência (600), o que representa, aproximadamente, 93% do total
#revision aparece somente 43 vezes
#isso indica que temos mais dados aprovados do que os que precisam de uma revisão

In [ ]:
#analisando a coluna 'true_class'
dados_analise_ml_semnull = dados_analise_ml.copy() #armazenando uma cópia do nosso dataframe em outra variável

#iremos fazer uma iteração em cada linha do nosso dataframe
for index, linha in dados_analise_ml_semnull.iterrows():
    
    #verificamos se o valor para 'True_class' é nan (nulo)
    if np.isnan(linha['True_class']):
        
        #se for nulo, iremos armazenar o valor do 'Pred_class' no seu lugar
        dados_analise_ml_semnull.loc[index, 'True_class'] = linha['Pred_class']

In [ ]:
#verificando se ainda existe algum valor nulo no dataframe e se a inserção funcionou
dados_analise_ml_semnull.isnull().sum()

In [ ]:
#verificando se inserimos os valores certos

#será criado um dataframe temporário para fazermos a comparação
#teremos duas colunas 'True_class': a da esquerda é antes de ser tratada e a direita depois de ser tratada.
#veremos que os valores foram inseridos corretamente
temp_data = pd.concat([dados_analise_ml, dados_analise_ml_semnull['True_class']], axis=1)
temp_data.tail(30)

In [ ]:
#deleta a variável temporária já que não usaremos ela para mais nada
del temp_data

In [ ]:
#usaremos só os dados onde o status é 'approved', pois foram confirmados e aprovados pelos cientistas
dados_analise_ml_semnull_approved = dados_analise_ml_semnull[dados_analise_ml_semnull['status'] == 'approved'].copy()

fig = plt.figure(figsize=(18, 12))
plt.title('Análise da coluna True_class')
sns.countplot(y=dados_analise_ml_semnull_approved['True_class'], orient='h')
fig.savefig('images/analise_true_class.png')

In [ ]:
#analisando o histograma da coluna 'True_class'
fig = plt.figure(figsize=(7, 4))
plt.title('Histograma da coluna True_class')
sns.histplot(dados_analise_ml_semnull_approved['True_class'])
fig.savefig('images/histograma_true_class.png')

In [ ]:
#temos um total de 69 classes diferentes
dados_analise_ml_semnull_approved['True_class'].nunique()

### 2- Calcule o desempenho do modelo de classificação utilizando pelo menos três métricas.

In [ ]:
#Por se tratar de um problema de classificação, as métricas utilizadas são voltadas para
#esse tipo de problema.
#Um problema de classificação consiste em atribuir uma classe, dentre todas as possibilidades possíveis,
#para uma determinada entrada.

#### Confusion Matrix

In [ ]:
confusion_matrix = confusion_matrix(dados_analise_ml_semnull_approved['True_class'],
                                        dados_analise_ml_semnull_approved['Pred_class'])

fig = plt.figure(figsize=(24, 18))
sns.heatmap(confusion_matrix, cmap='Blues')

#### Accuracy Score

In [ ]:
score = accuracy_score(dados_analise_ml_semnull_approved['True_class'],
                       dados_analise_ml_semnull_approved['Pred_class'])

score

#### F1 Score

In [ ]:
score = f1_score(dados_analise_ml_semnull_approved['True_class'],
                 dados_analise_ml_semnull_approved['Pred_class'],
                 average='weighted')

score

#### Recall Score

In [ ]:
score = recall_score(dados_analise_ml_semnull_approved['True_class'],
                     dados_analise_ml_semnull_approved['Pred_class'],
                     average='weighted')

score

### 3- Crie um classificador que tenha como output se os dados com status igual a revision estão corretos ou não (Sugestão : Técnica de cross-validation K-fold).

In [ ]:
#pega somente as linhas onde o status é 'revision'
dados_analise_ml_revision = dados_analise_ml_semnull[dados_analise_ml_semnull['status'] == 'revision'].copy()

dados_analise_ml_revision.head(10)

### 4- Compare três métricas de avaliação aplicadas ao modelo e descreva sobre a diferença.

### 5- Crie um classificador, a partir da segunda aba - NLP do arquivo de dados, quepermita identificar qual trecho de música corresponde às respectivas artistas listadas (Sugestão: Naive Bayes Classifier).

In [ ]:
#temos 518 dados e duas colunas
#colunas: letra e artista
#letra: trecho da música
#artista: cantora referente ao trecho da música (letra)
dados_nlp.shape

In [ ]:
#verificando se existe linhas com nulo
dados_nlp.describe()

In [ ]:
fig = plt.figure(figsize=(7, 4))
plt.title('Análise dos Artistas')
sns.countplot(dados_nlp['artista'])

#Temos dois artistas: Beyoncé e Rihanna
#Beyoncé aparece 274 vezes, aproximadamente 53% 
#Rihanna aparece 244 vezes, aproximadamente 47%

#### Criação do classificador

In [ ]:
#separando os dados em X(features) e y(target)

X = dados_nlp['letra']
y = dados_nlp['artista']

#separando os dados para treinamento do classificador e para testá-lo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=23)

#para a extração de features será utilizado o método bags of words
#o que esse método faz é associar um número inteiro para cada palavra
#e depois armazenará a quantidade de vezes que essa palavra apareceu.
#a função CountVectorizer já realiza todo esse processo de pré processamento e 
#tokenização (associar ao número inteiro) das palavras.
cv = CountVectorizer(stop_words='english')

X_train_tratado = cv.fit_transform(X_train)
X_test_tratado = cv.transform(X_test)

naive_bayes = MultinomialNB()

#treinando o classificador
naive_bayes.fit(X_train_tratado, y_train)

#testando o classificador com os dados de teste
pred = naive_bayes.predict(X_test_tratado)
np.mean(pred == y_test)